# Advent of Code #


In [1]:
# set up the environment
import numpy as np

## Day 1: Sonar Sweep ##
You're minding your own business on a ship at sea when the overboard alarm goes off! You rush to see if you can help. Apparently, one of the Elves tripped and accidentally sent the sleigh keys flying into the ocean!

Before you know it, you're inside a submarine the Elves keep ready for situations like this. It's covered in Christmas lights (because of course it is), and it even has an experimental antenna that should be able to track the keys if you can boost its signal strength high enough; there's a little meter that indicates the antenna's signal strength by displaying 0-50 stars.

In [2]:
# load the data from the source file 'data/depth.dat' and store it in an array.
depths = np.loadtxt("data/depths.dat", dtype="int", delimiter=",", unpack=False)
print(depths)
print(len(depths))

[ 173  175  171 ... 7118 7115 7121]
2000


### Problem 1 ##
As the submarine drops below the surface of the ocean, it automatically performs a sonar sweep of the nearby sea floor. On a small screen, the sonar sweep report (your puzzle input) appears: each line is a measurement of the sea floor depth as the sweep looks further and further away from the submarine.

In [3]:
depth_increase_count = 0
previous_depth = depths[0]
for x in depths:
    if x > previous_depth:
        depth_increase_count += 1
    previous_depth = x

print("Number of depth increase: ", depth_increase_count)

Number of depth increase:  1521


### Part 2 ###
Considering every single measurement isn't as useful as you expected: there's just too much noise in the data.

Instead, consider sums of a three-measurement sliding window. 

In [4]:
# create a new array of the sliding data
three_measurement_sums = []
for x in range(len(depths)-2):
    three_measurement_sums.append(depths[x]+depths[x+1]+depths[x+2])

# repeat the calculation with the above data
depth_increase_count = 0
previous_depth = three_measurement_sums[0]
for x in three_measurement_sums:
    if x > previous_depth:
        depth_increase_count += 1
    previous_depth = x

print("Number of depth increase: ", depth_increase_count)


Number of depth increase:  1543


## Day 2: Dive! ##
### Part One ###
It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

    forward X increases the horizontal position by X units.
    down X increases the depth by X units.
    up X decreases the depth by X units.

Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.

The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

forward 5
down 5
forward 8
up 3
down 8
forward 2

Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

    forward 5 adds 5 to your horizontal position, a total of 5.
    down 5 adds 5 to your depth, resulting in a value of 5.
    forward 8 adds 8 to your horizontal position, a total of 13.
    up 3 decreases your depth by 3, resulting in a value of 2.
    down 8 adds 8 to your depth, resulting in a value of 10.
    forward 2 adds 2 to your horizontal position, a total of 15.

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [5]:
# Import the manoeuvreing data into an array fo tuples 
manoeuvres = np.genfromtxt("data/manoeuvre.dat", dtype="U7, i4", delimiter=" ")
print(manoeuvres[:10], '...', manoeuvres[-10:])
print(len(manoeuvres))

[('forward', 8) ('forward', 9) ('forward', 9) ('down', 3) ('forward', 9)
 ('down', 1) ('down', 7) ('down', 7) ('down', 4) ('down', 2)] ... [('down', 8) ('forward', 6) ('forward', 7) ('forward', 9) ('forward', 4)
 ('down', 3) ('up', 5) ('down', 7) ('down', 7) ('forward', 9)]
1000


In [6]:
horizontal_pos = 0
depth = 0
for x in manoeuvres:
    direction = x[0]
    steps = x[1]
    if direction == "forward":
        horizontal_pos += steps
    if direction == "up":
        depth -= steps
    if direction == "down":
        depth += steps
    
print("Displacement: ", horizontal_pos, ", Depth: ", depth)
print("Product of displacement and dept: ", horizontal_pos * depth)


Displacement:  2003 , Depth:  872
Product of displacement and dept:  1746616


### Part Two ###
Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:

```
down X increases your aim by X units.
up X decreases your aim by X units.
forward X does two things:
    It increases your horizontal position by X units.
    It increases your depth by your aim multiplied by X.
```
Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.

Now, the above example does something different:

```
forward 5 adds 5 to your horizontal position, a total of 5. 
    Because your aim is 0, your depth does not change.
down 5 adds 5 to your aim, resulting in a value of 5.
forward 8 adds 8 to your horizontal position, a total of 13. 
    Because your aim is 5, your depth increases by 8*5=40.
up 3 decreases your aim by 3, resulting in a value of 2.
down 8 adds 8 to your aim, resulting in a value of 10.
forward 2 adds 2 to your horizontal position, a total of 15.
    Because your aim is 10, your depth increases by 2*10=20 to a total of 60.
```
After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course.

In [7]:
horizontal_pos = 0
depth = 0
aim = 0
for x in manoeuvres:
    direction = x[0]
    steps = x[1]
    if direction == "up":
        aim -= steps
    if direction == "down":
        aim += steps
    if direction == "forward":
        horizontal_pos += steps
        depth += aim * steps

    
print("Displacement: ", horizontal_pos, ", Depth: ", depth)
print("Product of displacement and dept: ", horizontal_pos * depth)

Displacement:  2003 , Depth:  869681
Product of displacement and dept:  1741971043


## Day 3: Binary Diagnostic ##
### Part 1 ###
The submarine has been making some odd creaking noises, so you ask it to produce a diagnostic report just in case.

The diagnostic report (your puzzle input) consists of a list of binary numbers which, when decoded properly, can tell you many useful things about the conditions of the submarine. The first parameter to check is the power consumption.

You need to use the binary numbers in the diagnostic report to generate two new binary numbers (called the gamma rate and the epsilon rate). The power consumption can then be found by multiplying the gamma rate by the epsilon rate.

Each bit in the gamma rate can be determined by finding the most common bit in the corresponding position of all numbers in the diagnostic report. For example, given the following diagnostic report:

```
00100
11110
10110
10111
10101
01111
00111
11100
10000
11001
00010
01010
```
Considering only the first bit of each number, there are five 0 bits and seven 1 bits. Since the most common bit is 1, the first bit of the gamma rate is 1.

The most common second bit of the numbers in the diagnostic report is 0, so the second bit of the gamma rate is 0.

The most common value of the third, fourth, and fifth bits are 1, 1, and 0, respectively, and so the final three bits of the gamma rate are 110.

So, the gamma rate is the binary number 10110, or 22 in decimal.

The epsilon rate is calculated in a similar way; rather than use the most common bit, the least common bit from each position is used. So, the epsilon rate is 01001, or 9 in decimal. Multiplying the gamma rate (22) by the epsilon rate (9) produces the power consumption, 198.

Use the binary numbers in your diagnostic report to calculate the gamma rate and epsilon rate, then multiply them together. What is the power consumption of the submarine? (Be sure to represent your answer in decimal, not binary.)

In [8]:
# Import the data into a text array - easier for getting positional data
diagnostics = np.genfromtxt("data/diagnostic.dat", dtype="U")
print(diagnostics[:10])
print(len(diagnostics))

['111010101100' '100001001100' '000111101100' '100100000000'
 '001001001110' '100110101011' '001001100101' '010000010110'
 '011011001001' '001001000101']
1000


In [9]:
# get the number of bits in the data
bits = len(diagnostics[0])
print(bits)

12


In [10]:
epsilon_str = ""
gamma_str = ""

for x in range(len(diagnostics[0])):
    gamma_val = 0
    for y in diagnostics:
        if y[x] == '1':
            gamma_val += 1
        else:
            gamma_val -= 1

# take advantage of the fact that the epsilon value == not(gamma value)
# (could be done later on the whole string but just as easy to build it now) 
    if gamma_val > 0:
        gamma_str += '1'
        epsilon_str +=  '0'
    else:
        gamma_str += '0'
        epsilon_str += '1'

# convert the base 2 strings to integers 
gamma_rate = int(gamma_str, 2)
epsilon_rate = int(epsilon_str, 2)

power_consumptions = gamma_rate * epsilon_rate

print("Gama rate: ", gamma_rate)
print("Epsilon rate: ", epsilon_rate)
print("Power Consumption: ", power_consumptions)


Gama rate:  3903
Epsilon rate:  192
Power Consumption:  749376


### Part Two ###

Next, you should verify the life support rating, which can be determined by multiplying the oxygen generator rating by the CO2 scrubber rating.

Both the oxygen generator rating and the CO2 scrubber rating are values that can be found in your diagnostic report - finding them is the tricky part. Both values are located using a similar process that involves filtering out values until only one remains. Before searching for either rating value, start with the full list of binary numbers from your diagnostic report and consider just the first bit of those numbers. Then:

    Keep only numbers selected by the bit criteria for the type of rating value for which you are searching.
        Discard numbers which do not match the bit criteria.
    If you only have one number left, stop; this is the rating value for which you are searching.
    Otherwise, repeat the process, considering the next bit to the right.

The bit criteria depends on which type of rating value you want to find:

To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

For example, to determine the oxygen generator rating value using the same example diagnostic report from above:

    Start with all 12 numbers and consider only the first bit of each number. 
        There are more 1 bits (7) than 0 bits (5)
        keep only the 7 numbers with a 1 in the first position:
            11110, 10110, 10111, 10101, 11100, 10000, and 11001.
    Then, consider the second bit of the 7 remaining numbers:
        there are more 0 bits (4) than 1 bits (3)
        keep only the 4 numbers with a 0 in the second position:
            10110, 10111, 10101, and 10000.
    In the third position,
        three of the four numbers have a 1
        keep those three:
            10110, 10111, and 10101.
    In the fourth position,
        two of the three numbers have a 1
        keep those two:
            10110 and 10111.
    In the fifth position,
        there are an equal number of 0 bits and 1 bits (one each).
        to find the oxygen generator rating, keep the number with a 1 in that position:
            10111.
    As there is only one number left, stop;
        the oxygen generator rating is 10111, or 23 in decimal.

Then, to determine the CO2 scrubber rating value from the same example above:

    Start again with all 12 numbers and consider only the first bit of each number.
        There are fewer 0 bits (5) than 1 bits (7)
        keep only the 5 numbers with a 0 in the first position:
            00100, 01111, 00111, 00010, and 01010.
    Then, consider the second bit of the 5 remaining numbers:
        there are fewer 1 bits (2) than 0 bits (3)
        keep only the 2 numbers with a 1 in the second position:
            01111 and 01010.
    In the third position, there are an equal number of 0 bits and 1 bits (one each).
        to find the CO2 scrubber rating, keep the number with a 0 in that position:
            01010.
    As there is only one number left, stop; 
        the CO2 scrubber rating is 01010, or 10 in decimal.

Finally, to find the life support rating, multiply the oxygen generator rating (23) by the CO2 scrubber rating (10) to get 230.

Use the binary numbers in your diagnostic report to calculate the oxygen generator rating and CO2 scrubber rating, then multiply them together. What is the life support rating of the submarine?

In [11]:
# create the arrays that will be reduced to a single value
o2_array = diagnostics.copy()
co2_array = diagnostics.copy()

def reduce_array(in_array: np.ndarray, check_greater: bool, bit_pos: int):
    
    # Returns the reduced array that includes only the required roww

    # Attributes
    # in_array:  numpy:np.ndarray
    #    the array whose rows are to be reduced
    # check_greater: bool
    #    are we checking for a greater preponderance of bits (O2 generator) or fewer (CO2 scrubber)
    # bit_pos: int
    #    the bit position to check
    
    total_1_bits = 0
    total_0_bits = 0
    for x in in_array:
        if x[bit_pos] == '1':
            total_1_bits += 1
        else:
            total_0_bits += 1

    temp_array: np.ndarray = []
    for y in in_array:
        bit_value = y[bit_pos]
        if check_greater:
            want_1s = True if total_1_bits >= total_0_bits else False
            if want_1s and bit_value == '1':
                temp_array.append(y)
            else:
                if not want_1s and bit_value == '0':
                    temp_array.append(y)
        else:
            want_0s = True if total_0_bits <= total_1_bits else False
            if want_0s and bit_value =='0':
                temp_array.append(y)
            else:
                if not want_0s and bit_value =='1':
                    temp_array.append(y)

    return temp_array

number_of_bits = len(o2_array[0])

check_bit = 0
while check_bit < number_of_bits:
    if len(o2_array) > 1:
        # check the Oxygen Generator - pick the greatest bit value for retaining
        o2_array =  reduce_array(o2_array, True, check_bit)
    if len(co2_array) >1:
        # check the CO2 Scrubber - pick the least bit value for retaining
        co2_array = reduce_array(co2_array, False, check_bit)
    check_bit += 1

  
# convert the reduced arrays to integers
o2_generator_rating = int(o2_array[0], 2)
co2_scrubber_rating = int(co2_array[0], 2)

life_support_rating = o2_generator_rating * co2_scrubber_rating

print("O2 Generator rating: ", o2_generator_rating)
print("CO2 Scrubber rating: ", co2_scrubber_rating)

print("Life Support rating: ", life_support_rating)


O2 Generator rating:  3871
CO2 Scrubber rating:  613
Life Support rating:  2372923


## Day 4: Giant Squid ##

### Part 1 ###
You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you can see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play bingo?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is marked on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board wins. (Diagonals don't count.)

The submarine has a bingo subsystem to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

`7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1`

```
22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7
```
After the first five numbers are drawn `(7, 4, 9, 5, and 11)`, there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
After the next six numbers are drawn `(17, 23, 2, 0, 14, and 21)`, there are still no winners:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
Finally, `24` is drawn:
```
22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7
```
At this point, the third board wins because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: `14 21 17 24 4`).

The score of the winning board can now be calculated. Start by finding the sum of all unmarked numbers on that board; in this case, the sum is 188. Then, multiply that sum by the number that was just called when the board won, 24, to get the final score, 188 * 24 = 4512.

To guarantee victory against the giant squid, figure out which board will win first. What will your final score be if you choose that board?

### Inport the data and define a bingo card ###

In [20]:
# Import the data into array 
bingo_calls = np.genfromtxt("data/bingo_calls.dat", dtype="i4", delimiter=',')
raw_cards = np.genfromtxt("data/bingo_cards.dat", dtype="i4").reshape(100,5,5)

# define a bingo_card
class BingoCard(object):
    # A bingo card consists of a 6 x 6 matrix
    # rows 1 - 5 and columns 1 - 5 contain the cards numbers
    # row 6 column 1 - 5 contains the count of the numbers called in that column
    # column 6  rows 1 -5 contains the count of the numbers called in that row
    # row 6 column 6 indicates a winning card - if any of row 6 or column 6 reaches 5

    def __init__(self) -> None:
        self.card = np.zeros((6,6), dtype="i4")
        self.totals = 0
        self.card_no = 0
        self.last_call = -1
        self.__has_won = False
        self.no_of_checks = 0
        

    def __str__(self) -> str:
        card = f'{self.card[:]}'
        return card

    def build(self, lines:np.ndarray, card_no: int) -> None:
        self.totals = np.sum(lines)
        self.card = np.column_stack((lines, [5, 5, 5, 5, 5]))
        self.card = np.row_stack((self.card, [5, 5, 5, 5, 5, 0]))
        self.card_no = card_no

    def check_number(self, number) -> bool:
        if self.__has_won:
            return True
        self.no_of_checks += 1
        res = np.where(self.card[:5, :5] == number)
        if res[0].size == 0:
            return False
        self.last_call = number
        row = res[0]
        col = res[1]
        self.card[row,5] -= 1
        self.card[5,col] -= 1
        self.card[row, col] *= -1
        self. totals -= number
        if self.card[row,5] == 0 or self.card[5, col] == 0:
            self.__has_won = True
            return True

        return False

    def unchecked_sum(self) -> int:
        return self.totals

    def is_winner(self) -> bool:
        return self.__has_won
    


### Set up the board ##

In [21]:
def set_bingo_cards() -> np.ndarray:
    cards = []
    for x, raw_card in enumerate(raw_cards):
        card = BingoCard()
        card.build(raw_card, x+1)
        cards.append(card)
    return cards



### Check for the first winning card ###

In [22]:
bingo_cards = set_bingo_cards()

found_winner = False
winning_card = None
last_call = -1
for call in bingo_calls:
    last_call = call
    for card in bingo_cards:
        if card.check_number(call):
            found_winner = True
            winning_card = card
            break
    if found_winner:
        break

if winning_card :
    unchecked_score = winning_card.unchecked_sum()
    print('Card {} is a winner!\n{}'.format(winning_card.card_no, winning_card))
    print('Last call was {} and the total of uncheched numbers is {}'.format(
        winning_card.last_call, unchecked_score
    ))
    print('Final Score: {}'.format(unchecked_score * last_call))


Card 50 is a winner!
[[-15  56 -39  57  40   3]
 [-67 -59  26  30  90   3]
 [-84  -2 -41 -25  -7   0]
 [ 96 -23  79  99  85   4]
 [ 13  10  86  51  53   5]
 [  2   2   3   4   4   0]]
Last call was 41 and the total of uncheched numbers is 871
Final Score: 35711


### Reset the board and find the last winning card ###

This time play all the numbers in the call list.  At the end we check each ***winning*** card and the one with the highest number of checks before winning is the one that is required. 

In [27]:
bingo_cards = set_bingo_cards()

last_call_ordinal = -1
bingo_card = None

for call in bingo_calls:
    for card in bingo_cards:
        card.check_number(call)

winning_card = None
max_calls = 0
for card in bingo_cards:
    if card.is_winner():
        card_checks = card.no_of_checks
        if card_checks > max_calls:
            max_calls = card_checks
            winning_card = card
    

if winning_card:
    unchecked_score = winning_card.unchecked_sum()
    last_call = winning_card.last_call
    print('Card {}\n{}\nis the last winner!'.format(winning_card.card_no, winning_card))
    print('Last call was {} and the total of uncheched numbers is {}'.format(
        last_call, unchecked_score
    ))
    print('Final Score: {}'.format(unchecked_score * last_call))

Card 31
[[  9 -93 -65 -94 -29   1]
 [ -2  80  -7  16 -79   2]
 [-11  -5 -21  73 -50   1]
 [-20 -70 -37 -48  85   1]
 [-99   3 -55 -58  -8   1]
 [  1   2   0   2   1   0]]
is the last winner!
Last call was 21 and the total of uncheched numbers is 266
Final Score: 5586


Day 5: Hydrothermal Venture ---

You come across a field of hydrothermal vents on the ocean floor! These vents constantly produce large, opaque clouds, so it would be best to avoid them if possible.

They tend to form in lines; the submarine helpfully produces a list of nearby lines of vents (your puzzle input) for you to review. For example:

0,9 -> 5,9
8,0 -> 0,8
9,4 -> 3,4
2,2 -> 2,1
7,0 -> 7,4
6,4 -> 2,0
0,9 -> 2,9
3,4 -> 1,4
0,0 -> 8,8
5,5 -> 8,2

Each line of vents is given as a line segment in the format x1,y1 -> x2,y2 where x1,y1 are the coordinates of one end the line segment and x2,y2 are the coordinates of the other end. These line segments include the points at both ends. In other words:

    An entry like 1,1 -> 1,3 covers points 1,1, 1,2, and 1,3.
    An entry like 9,7 -> 7,7 covers points 9,7, 8,7, and 7,7.

For now, only consider horizontal and vertical lines: lines where either x1 = x2 or y1 = y2.

So, the horizontal and vertical lines from the above list would produce the following diagram:

.......1..
..1....1..
..1....1..
.......1..
.112111211
..........
..........
..........
..........
222111....

In this diagram, the top left corner is 0,0 and the bottom right corner is 9,9. Each position is shown as the number of lines which cover that point or . if no line covers that point. The top-left pair of 1s, for example, comes from 2,2 -> 2,1; the very bottom row is formed by the overlapping lines 0,9 -> 5,9 and 0,9 -> 2,9.

To avoid the most dangerous areas, you need to determine the number of points where at least two lines overlap. In the above example, this is anywhere in the diagram with a 2 or larger - a total of 5 points.

Consider only horizontal and vertical lines. At how many points do at least two lines overlap?